# Irina Viner-Usmanova Rhythmic Gymnastics Centre

In [29]:
using WebIO

In [30]:
using Interact

In [31]:
using Khepri

In [32]:
render_size(800, 400) # Khepri backends size

(800, 400)

## Sinusoidal Curves

 | Sinunoisal wave parameters | Other sinusoidal curves |
 |----------------------------|-------------------------|
 |<img src="./Figures/sin_wave.png" width="350"> | <img src="./Figures/sin_curves.png" width="350">|

`sinusoidal` parameters explained:
* a is the amplitude
* omega is the frequency
* fi is the phase

In [33]:
sinusoidal(a, omega, fi, x) = a*sin(omega*x+fi)

sinusoidal (generic function with 1 method)

In [34]:
sin_array_y(p, a, omega, fi, dist, n) = [p+vxy(i, sinusoidal(a, omega, fi, i)) for i in division(0, dist, n)]

sin_array_y (generic function with 1 method)

In [35]:
backend(notebook)
new_backend()

line(sin_array_y(u0(), 5, 1, 0, 50, 100))

data: []

layout: "layout with fields autosize, height, margin, scene, showlegend, and width"

Line(...)

`damped_sin_wave`'s parameters explained
* a is the initial amplitude (the highest peak)
* b is the decay constant
* omega is the angular frequency

In [36]:
damped_sin_wave(a, d, omega, x) = a*exp(-(d*x))*sin(omega*x)

damped_sin_wave (generic function with 1 method)

In [37]:
damped_sin_array_z(p, a, d, omega, dist, n) = [p+vxz(i, damped_sin_wave(a, d, omega, i)) for i in division(0, dist, n)]

damped_sin_array_z (generic function with 1 method)

In [38]:
backend(notebook)
new_backend()

line(damped_sin_array_z(u0(), 5, 0.1, 1, 50, 100))

data: []

layout: "layout with fields autosize, height, margin, scene, showlegend, and width"

Line(...)

In [39]:
damped_sin_roof_pts(p, a_x, a_y, fi, decay, om_x, om_y, dist_x, dist_y, n_x, n_y) =
    map_division((x, y)->p+vxyz(x, y, sinusoidal(a_x, om_x, fi, x)+damped_sin_wave(a_y, decay, om_y, y)),
        0, dist_x, n_x,
        0, dist_y, n_y)

damped_sin_roof_pts (generic function with 2 methods)

In [40]:
backend(meshcat)
new_backend()

@manipulate for ampx = widget(0:0.1:3, label="Amplitude x"),
                ampy = widget(0:0.1:20, label="Amplitude y")
            delete_all_shapes()
            surface_grid(damped_sin_roof_pts(u0(), ampx, ampy, 0, 0.1, 0.2, 1, 50, 100, 100, 200))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8704
└ @ MeshCat C:\Users\ines\.julia\packages\MeshCat\ECbzr\src\visualizer.jl:73


Khepri.MCATViewer(MeshCat Visualizer with path /meshcat at http://localhost:8704)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Amplitude x"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 31,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
16, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000000177319f0, Task (runnable) @0x00000000177319f0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.1\",\"0.2\",\"0.3\",\"0.4\",\"0.5\",\"0.6\",\"0.7\",\"0.8\",\"0.9\",\"1.0\",\"1.1\",\"1.2\",\"1.3\",\"1.4\",\"1.5\",\"1.6\",\"1.7\",\"1.8\",\"1.9\",\"2.0\",\"2.1\",\"2.2\",\

In [41]:
damped_sin_roof_pts(p, h, a_x, a_y_min, a_y_max, fi, decay, om_x, om_y, dist_x, dist_y, n_x, n_y) =
    map_division((x, y) ->
                y <= d_i ?
                p + vxyz(x, 
                         -sin(y/d_i*(1*pi)), 
                         y*h/d_i + sin(x/dist_x*pi)sinusoidal(a_x, om_x, fi-y*pi/dist_y, x)*(y*a_x/d_i)) :
                p + vxyz(x,
                         y,
                         h + sin(x/dist_x*pi)*sinusoidal(a_x, om_x, fi-y*pi/dist_y, x) +
                         damped_sin_wave(a_y_max - (a_y_max-a_y_min)/dist_x*x, decay, om_y, y)),
             0, dist_x, n_x,
             0, dist_y, n_y)

damped_sin_roof_pts (generic function with 2 methods)

In [42]:
backend(meshcat)
new_backend()
d_i = 0.5

@manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                ampy_min = widget(1:0.1:15, label="Minimum Amplitude y"),
                ampy_max = widget(1:0.1:15, label="Maximum Amplitude y"),
                h = widget(2:1:20, label="Height")
            delete_all_shapes()
            surface_grid(damped_sin_roof_pts(u0(), h, ampx, ampy_min, ampy_max, pi, 0.03, pi/50, pi/10, 60, 100, 120, 800))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8705
└ @ MeshCat C:\Users\ines\.julia\packages\MeshCat\ECbzr\src\visualizer.jl:73


Khepri.MCATViewer(MeshCat Visualizer with path /meshcat at http://localhost:8705)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Amplitude x"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 21,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
11, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000014996570, Task (runnable) @0x0000000014996570), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1.0\",\"1.1\",\"1.2\",\"1.3\",\"1.4\",\"1.5\",\"1.6\",\"1.7\",\"1.8\",\"1.9\",\"2.0\",\"2.1\",\"2.2\",\"2.3\",\"2.4\",\"2.5\",\"2.6\",\"2.7\",\"2.8\",\"2.9\",\"3.0\"],\"changes\":WebI

## Pavilion Dimensions

In [44]:
amp_x = 2.5
amp_y_min_top = 4
amp_y_max_top = 11
amp_y_max_bottom = 10
amp_y_min_bottom = 3
fi = pi
decay = 0.03
om_x = pi/50
om_y = pi/10
pav_width = 60
pav_length = 100
pav_height = 14
d_width = 1.5
d_length = 1
d_height = 3

3

## Roof Truss

The following functions create truss elements:
* `free_node` and `fixed_node`create truss nodes
* `bar`creates truss bars
* `panel`creates truss panels

In [46]:
free_node(pt) =
    sphere(pt, 0.1)

free_node (generic function with 1 method)

In [47]:
fixed_node(pt) =
    sphere(pt, 0.1)

fixed_node (generic function with 1 method)

In [48]:
bar(p, q) =
    truss_bar(p, q)

bar (generic function with 1 method)

In [49]:
truss_panel(ps) =
    surface_polygon(ps)

truss_panel (generic function with 1 method)

`truss_ptss`receives two sets of points, the bottom and top points defining the shape of the truss to create.
It returns the points defining the truss structure.

In [50]:
truss_ptss(bottom_ptss,top_ptss) =
    length(top_ptss) == length(bottom_ptss) ?
    vcat([[pts,qts] for (pts,qts) in zip(bottom_ptss,top_ptss)]...) :
    vcat([[pts,qts] for (pts,qts) in zip(top_ptss[1:end-1],bottom_ptss)]..., [top_ptss[end]])

truss_ptss (generic function with 1 method)

`truss`receives the type of truss to create, as well as both bottom and top points defining its shape.
It creates a 3D spatial truss.

In [51]:
truss(truss_type; bottom_ptss=planar(p=x(0)), top_ptss=planar(p=z(1))) =
    let ptss = truss_ptss(bottom_ptss, top_ptss)
      truss_type(ptss)
  end

truss (generic function with 1 method)

`vierendeel`creates a vierendeel truss structure.

In [52]:
vierendeel(ptss, first=true) =
    let ais = ptss[1],
        bis = ptss[2],
        cis = ptss[3],
        dis = ptss[4]
      (first ? fixed_node : free_node).(ais)
      free_node.(bis)
      bar.(ais, bis)
      bar.(ais, cis)
      bar.(bis, dis)
      bar.(ais[2:end], ais[1:end-1])
      bar.(bis[2:end], bis[1:end-1])
      if ptss[5:end] == []
          fixed_node.(cis)
          free_node.(dis)
          bar.(cis[2:end], cis[1:end-1])
          bar.(dis[2:end], dis[1:end-1])
          bar.(dis, cis)
      else
          vierendeel(ptss[3:end], false)
      end
    end

vierendeel (generic function with 2 methods)

In [53]:
backend(meshcat)
new_backend()

@manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                ampy_top_min = widget(3:0.1:15, label="Minimum Amplitude y"),
                ampy_top_max = widget(3:0.1:15, label="Maximum Amplitude y"),
                ampy_bottom_min = widget(1:0.1:12, label="Minimum Amplitude y"),
                ampy_bottom_max = widget(1:0.1:12, label="Maximum Amplitude y"),
                h = widget(2:1:20, label="Height")
            delete_all_shapes()
    truss(vierendeel, 
      top_ptss=damped_sin_roof_pts(u0(), pav_height,
                                      ampx, ampy_top_min, ampy_top_max,
                                      fi, decay, om_x, om_y,
                                      pav_width, pav_length-d_length, 50, 80),
      bottom_ptss=damped_sin_roof_pts(xy(d_width,d_length*1), pav_height-d_height,
                                     ampx, ampy_bottom_min, ampy_bottom_max,
                                     fi, decay, om_x, om_y,
                                     pav_width - d_width*2, pav_length-2*d_length, 50, 80))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8707
└ @ MeshCat C:\Users\ines\.julia\packages\MeshCat\ECbzr\src\visualizer.jl:73


Khepri.MCATViewer(MeshCat Visualizer with path /meshcat at http://localhost:8707)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Amplitude x"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 21,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
11, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000016c65430, Task (runnable) @0x0000000016c65430), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1.0\",\"1.1\",\"1.2\",\"1.3\",\"1.4\",\"1.5\",\"1.6\",\"1.7\",\"1.8\",\"1.9\",\"2.0\",\"2.1\",\"2.2\",\"2.3\",\"2.4\",\"2.5\",\"2.6\",\"2.7\",\"2.8\",\"2.9\",\"3.0\"],\"changes\":WebI

## Roof Surface

## Glass Façade

In [43]:
damped_sin_glass_wall(p, a_y, fi, decay, om_y, dist_y, dist_z, n_y, n_z) =
    map_division((y, z) ->
                p + vyz(y,
                        z + (z/dist_z)*damped_sin_wave(a_y, decay, om_y, y)),
             0, dist_y, n_y,
             0, dist_z, n_z)

damped_sin_glass_wall (generic function with 1 method)

## Interior Walls